# T2. databundle 和 tokenizer 的基本使用

&emsp; 1 &ensp; fastNLP 中 dataset 的延伸

&emsp; &emsp; 1.1 &ensp; databundle 的概念与使用

&emsp; 2 &ensp; fastNLP 中的 tokenizer
 
&emsp; &emsp; 2.1 &ensp; PreTrainedTokenizer 的概念

&emsp; &emsp; 2.2 &ensp; BertTokenizer 的基本使用
 
&emsp; &emsp; 2.3 &ensp; 补充：GloVe 词嵌入的使用

## 1. fastNLP 中 dataset 的延伸

### 1.1 databundle 的概念与使用

在`fastNLP 0.8`中，在常用的数据加载模块`DataLoader`和数据集`DataSet`模块之间，还存在

&emsp; 一个中间模块，即 **数据包`DataBundle`模块**，可以从`fastNLP.io`路径中导入该模块

在`fastNLP 0.8`中，**一个`databundle`数据包包含若干`dataset`数据集和`vocabulary`词汇表**

&emsp; 分别存储在`datasets`和`vocabs`两个变量中，所以了解`databundle`数据包之前

需要首先**复习`dataset`数据集和`vocabulary`词汇表**，**下面的一串代码**，**你知道其大概含义吗？**

<!-- 必要提示：`NG20`，全称[`News Group 20`](http://qwone.com/~jason/20Newsgroups/)，是一个新闻文本分类数据集，包含20个类别

&emsp; 数据集包含训练集`'ng20_train.csv'`和测试集`'ng20_test.csv'`两部分，每条数据

&emsp; 包括`'label'`标签和`'text'`文本两个条目，通过`sample(frac=1)[:6]`随机采样并读取前6条 -->

In [1]:
import pandas as pd

from fastNLP import DataSet
from fastNLP import Vocabulary
from fastNLP.io import DataBundle

datasets = DataSet.from_pandas(pd.read_csv('./data/test4dataset.tsv', sep='\t'))
datasets.rename_field('Sentence', 'text')
datasets.rename_field('Sentiment', 'label')
datasets.apply_more(lambda ins:{'label': ins['label'].lower(), 
                                'text': ins['text'].lower().split()},
                    progress_bar='tqdm')
datasets.delete_field('SentenceId')
train_ds, test_ds = datasets.split(ratio=0.7)
datasets = {'train': train_ds, 'test': test_ds}
print(datasets['train'])
print(datasets['test'])

vocabs = {}
vocabs['label'] = Vocabulary().from_dataset(datasets['train'].concat(datasets['test'], inplace=False), field_name='label')
vocabs['text'] = Vocabulary().from_dataset(datasets['train'].concat(datasets['test'], inplace=False), field_name='text')
print(vocabs['label'].word2idx)

Processing:   0%|          | 0/6 [00:00<?, ?it/s]

+------------------------------------------+----------+
| text                                     | label    |
+------------------------------------------+----------+
| ['this', 'quiet', ',', 'introspective... | positive |
| ['a', 'comedy-drama', 'of', 'nearly',... | positive |
| ['a', 'positively', 'thrilling', 'com... | neutral  |
| ['a', 'series', 'of', 'escapades', 'd... | negative |
+------------------------------------------+----------+
+------------------------------------------+----------+
| text                                     | label    |
+------------------------------------------+----------+
| ['even', 'fans', 'of', 'ismail', 'mer... | negative |
| ['the', 'importance', 'of', 'being', ... | neutral  |
+------------------------------------------+----------+
{'<pad>': 0, '<unk>': 1, 'positive': 2, 'neutral': 3, 'negative': 4}


<!-- 上述代码的含义是：随机读取`NG20`数据集中的各十条训练数据和测试数据，将标签都设为小写，对文本进行分词
 -->
上述代码的含义是：从`test4dataset`的 6 条数据中，划分 4 条训练集（`int(6*0.7) = 4`），2 条测试集

&emsp; &emsp; 修改相关字段名称，删除序号字段，同时将标签都设为小写，对文本进行分词

&emsp; 接着通过`concat`方法拼接测试集训练集，注意设置`inplace=False`，生成临时的新数据集

&emsp; 使用`from_dataset`方法从拼接的数据集中抽取词汇表，为将数据集中的单词替换为序号做准备

由此就可以得到**数据集字典`datasets`**（**对应训练集、测试集**）和**词汇表字典`vocabs`**（**对应数据集各字段**）

&emsp; 然后就可以初始化`databundle`了，通过`print`可以观察其大致结构，效果如下

In [2]:
data_bundle = DataBundle(datasets=datasets, vocabs=vocabs)
print(data_bundle)
print(data_bundle.get_dataset_names())
print(data_bundle.get_vocab_names())

In total 2 datasets:
	train has 4 instances.
	test has 2 instances.
In total 2 vocabs:
	label has 5 entries.
	text has 96 entries.

['train', 'test']
['label', 'text']


此外，也可以通过`data_bundle`的`num_dataset`和`num_vocab`返回数据表和词汇表个数

&emsp; 通过`data_bundle`的`iter_datasets`和`iter_vocabs`遍历数据表和词汇表

In [3]:
print("In total %d datasets:" % data_bundle.num_dataset)
for name, dataset in data_bundle.iter_datasets():
    print("\t%s has %d instances." % (name, len(dataset)))
print("In total %d datasets:" % data_bundle.num_dataset)
for name, vocab in data_bundle.iter_vocabs():
    print("\t%s has %d entries." % (name, len(vocab)))

In total 2 datasets:
	train has 4 instances.
	test has 2 instances.
In total 2 datasets:
	label has 5 entries.
	text has 96 entries.


在数据包`databundle`中，也有和数据集`dataset`类似的四个`apply`函数，即

&emsp; `apply`函数、`apply_field`函数、`apply_field_more`函数和`apply_more`函数

&emsp; 负责对数据集进行预处理，如下所示是`apply_more`函数的示例，其他函数类似

此外，通过`get_dataset`函数，可以通过数据表名`name`称找到对应数据表

&emsp; 通过`get_vocab`函数，可以通过词汇表名`field_name`称找到对应词汇表

In [4]:
data_bundle.apply_more(lambda ins:{'len': len(ins['text'])}, progress_bar='tqdm')
print(data_bundle.get_dataset('train'))

Processing:   0%|          | 0/4 [00:00<?, ?it/s]

Processing:   0%|          | 0/2 [00:00<?, ?it/s]

+------------------------------+----------+-----+
| text                         | label    | len |
+------------------------------+----------+-----+
| ['this', 'quiet', ',', 'i... | positive | 11  |
| ['a', 'comedy-drama', 'of... | positive | 19  |
| ['a', 'positively', 'thri... | neutral  | 26  |
| ['a', 'series', 'of', 'es... | negative | 37  |
+------------------------------+----------+-----+


## 2. fastNLP 中的 tokenizer

### 2.1 PreTrainTokenizer 的提出

为什么要放弃传统的GloVe词嵌入？

在`fastNLP 0.8`中，**使用`PreTrainedTokenizer`模块来为数据集中的词语进行词向量的标注**

&emsp; 需要注意的是，`PreTrainedTokenizer`模块的下载和导入**需要确保环境安装了`transformers`模块**

&emsp; 这是因为 `fastNLP 0.8`中`PreTrainedTokenizer`模块的实现基于`Huggingface Transformers`库

**`Huggingface Transformers`是基于一个开源的**，**基于`transformer`模型结构提供的预训练语言库**

&emsp; 包含了多种经典的基于`transformer`的预训练模型，如`BERT`、`BART`、`RoBERTa`、`GPT2`、`CPT`

&emsp; 更多相关内容可以参考`Huggingface Transformers`的[相关论文](https://arxiv.org/pdf/1910.03771.pdf)、[官方文档](https://huggingface.co/transformers/)以及[的代码仓库](https://github.com/huggingface/transformers)

### 2.2 BertTokenizer 的基本使用

在`fastNLP 0.8`中，以`PreTrainedTokenizer`为基类，泛化出多个子类，实现基于`BERT`等模型的标注

&emsp; 本节以`BertTokenizer`模块为例，展示`PreTrainedTokenizer`模块的使用方法与应用实例

**`BertTokenizer`的初始化包括 导入模块和导入数据 两步**，先通过从`fastNLP.transformers.torch`中

&emsp; 导入`BertTokenizer`模块，再**通过`from_pretrained`方法指定`tokenizer`参数类型下载**

&emsp; 其中，**`'bert-base-uncased'`指定`tokenizer`使用的预训练`BERT`类型**：单词不区分大小写

&emsp; &emsp; **模块层数`L=12`**，**隐藏层维度`H=768`**，**自注意力头数`A=12`**，**总参数量`110M`**

&emsp; 另外，模型参数自动下载至 home 目录下的`~\.cache\huggingface\transformers`文件夹中

In [5]:
from fastNLP.transformers.torch import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

通过变量`vocab_size`和`vocab_files_names`可以查看`BertTokenizer`的词汇表的大小和对应文件

&emsp; 通过变量`vocab`可以访问`BertTokenizer`预训练的词汇表（由于内容过大就不演示了

In [6]:
print(tokenizer.vocab_size, tokenizer.vocab_files_names)

30522 {'vocab_file': 'vocab.txt'}


通过变量`all_special_tokens`或通过变量`special_tokens_map`可以**查看`BertTokenizer`内置的特殊词素**

&emsp; 包括**未知符`'[UNK]'`**, **断句符`'[SEP]'`**, **补零符`'[PAD]'`**, **分类符`'[CLS]'`**, **掩模`'[MASK]'`**

通过变量`all_special_ids`可以**查看`BertTokenizer`内置的特殊词素对应的词汇表编号**，相同功能

&emsp; 也可以直接通过查看`pad_token`，值为`'[UNK]'`，和`pad_token_id`，值为`0`，等变量来实现

In [7]:
print('pad_token', tokenizer.pad_token, tokenizer.pad_token_id) 
print('unk_token', tokenizer.unk_token, tokenizer.unk_token_id) 
print('cls_token', tokenizer.cls_token, tokenizer.cls_token_id) 
print('sep_token', tokenizer.sep_token, tokenizer.sep_token_id)
print('msk_token', tokenizer.mask_token, tokenizer.mask_token_id)
print('all_tokens', tokenizer.all_special_tokens, tokenizer.all_special_ids)
print(tokenizer.special_tokens_map)

pad_token [PAD] 0
unk_token [UNK] 100
cls_token [CLS] 101
sep_token [SEP] 102
msk_token [MASK] 103
all_tokens ['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]'] [100, 102, 0, 101, 103]
{'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}


此外，还可以添加其他特殊字符，例如起始符`[BOS]`、终止符`[EOS]`，添加后词汇表编号也会相应改变

&emsp; *但是如何添加这两个之外的字符，并且如何将这两个的编号设置为`[UNK]`之外的编号？？？*

In [8]:
tokenizer.bos_token = '[BOS]'
tokenizer.eos_token = '[EOS]'
# tokenizer.bos_token_id = 104
# tokenizer.eos_token_id = 105
print('bos_token', tokenizer.bos_token, tokenizer.bos_token_id)
print('eos_token', tokenizer.eos_token, tokenizer.eos_token_id)
print('all_tokens', tokenizer.all_special_tokens, tokenizer.all_special_ids)
print(tokenizer.special_tokens_map)

bos_token [BOS] 100
eos_token [EOS] 100
all_tokens ['[BOS]', '[EOS]', '[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]'] [100, 100, 100, 102, 0, 101, 103]
{'bos_token': '[BOS]', 'eos_token': '[EOS]', 'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}


在`BertTokenizer`中，**使用`tokenize`函数和`convert_tokens_to_string`函数可以实现文本和词素列表的互转**

&emsp; 此外，**使用`convert_tokens_to_ids`函数和`convert_ids_to_tokens`函数则可以实现词素和词素编号的互转**

&emsp; 上述四个函数的使用效果如下所示，此处可以明显看出，`tokenizer`分词和传统分词的不同效果，例如`'##cap'`

In [9]:
text = "a series of escapades demonstrating the adage that what is " \
       "good for the goose is also good for the gander , some of which " \
       "occasionally amuses but none of which amounts to much of a story ." 
tks = ['a', 'series', 'of', 'es', '##cap', '##ades', 'demonstrating', 'the', 
       'ada', '##ge', 'that', 'what', 'is', 'good', 'for', 'the', 'goose', 
       'is', 'also', 'good', 'for', 'the', 'gan', '##der', ',', 'some', 'of', 
       'which', 'occasionally', 'am', '##uses', 'but', 'none', 'of', 'which', 
       'amounts', 'to', 'much', 'of', 'a', 'story', '.']
ids = [ 1037,  2186,  1997,  9686, 17695, 18673, 14313,  1996, 15262,  3351, 
        2008,  2054,  2003,  2204,  2005,  1996, 13020,  2003,  2036,  2204,
        2005,  1996, 25957,  4063,  1010,  2070,  1997,  2029,  5681,  2572,
       25581,  2021,  3904,  1997,  2029,  8310,  2000,  2172,  1997,  1037,
        2466,  1012]

tokens = tokenizer.tokenize(text)
print(tokenizer.convert_tokens_to_ids(tokens))

ids = tokenizer.convert_tokens_to_ids(tokens)
print(tokenizer.convert_ids_to_tokens(ids))

print(tokenizer.convert_tokens_to_string(tokens))

[1037, 2186, 1997, 9686, 17695, 18673, 14313, 1996, 15262, 3351, 2008, 2054, 2003, 2204, 2005, 1996, 13020, 2003, 2036, 2204, 2005, 1996, 25957, 4063, 1010, 2070, 1997, 2029, 5681, 2572, 25581, 2021, 3904, 1997, 2029, 8310, 2000, 2172, 1997, 1037, 2466, 1012]
['a', 'series', 'of', 'es', '##cap', '##ades', 'demonstrating', 'the', 'ada', '##ge', 'that', 'what', 'is', 'good', 'for', 'the', 'goose', 'is', 'also', 'good', 'for', 'the', 'gan', '##der', ',', 'some', 'of', 'which', 'occasionally', 'am', '##uses', 'but', 'none', 'of', 'which', 'amounts', 'to', 'much', 'of', 'a', 'story', '.']
a series of escapades demonstrating the adage that what is good for the goose is also good for the gander , some of which occasionally amuses but none of which amounts to much of a story .


在`BertTokenizer`中，还有另外两个函数可以实现分词标注，分别是 **`encode`和`decode`函数**，**可以直接实现**

&emsp; **文本字符串和词素编号列表的互转**，但是编码过程中会按照`BERT`的规则，**在句子首末加入`[CLS]`和`[SEP]`**

In [10]:
enc = tokenizer.encode(text)
print(tokenizer.encode(text))
dec = tokenizer.decode(enc)
print(tokenizer.decode(enc))

[101, 1037, 2186, 1997, 9686, 17695, 18673, 14313, 1996, 15262, 3351, 2008, 2054, 2003, 2204, 2005, 1996, 13020, 2003, 2036, 2204, 2005, 1996, 25957, 4063, 1010, 2070, 1997, 2029, 5681, 2572, 25581, 2021, 3904, 1997, 2029, 8310, 2000, 2172, 1997, 1037, 2466, 1012, 102]
[CLS] a series of escapades demonstrating the adage that what is good for the goose is also good for the gander, some of which occasionally amuses but none of which amounts to much of a story. [SEP]


在`encode`函数之上，还有`encode_plus`函数，这也是在数据预处理中，`BertTokenizer`模块最常用到的函数

&emsp; **`encode`函数的参数**，**`encode_plus`函数都有**；**`encode`函数词素编号列表**，**`encode_plus`函数返回字典**

在`encode_plus`函数的返回值中，字段`input_ids`表示词素编号，其余两个字段后文有详细解释

&emsp; **字段`token_type_ids`详见`text_pairs`的示例**，**字段`attention_mask`详见`batch_text`的示例**

在`encode_plus`函数的参数中，参数`add_special_tokens`表示是否按照`BERT`的规则，加入相关特殊字符

&emsp; 参数`max_length`表示句子截取最大长度（算特殊字符），在参数`truncation=True`时会自动截取

&emsp; 参数`return_attention_mask`约定返回的字典中是否包括`attention_mask`字段，以上案例如下

In [11]:
text = "a series of escapades demonstrating the adage that what is good for the goose is also good for "\
       "the gander , some of which occasionally amuses but none of which amounts to much of a story ." 

encoded = tokenizer.encode_plus(text=text, add_special_tokens=True, max_length=32, 
                                truncation=True, return_attention_mask=True)
print(encoded)

{'input_ids': [101, 1037, 2186, 1997, 9686, 17695, 18673, 14313, 1996, 15262, 3351, 2008, 2054, 2003, 2204, 2005, 1996, 13020, 2003, 2036, 2204, 2005, 1996, 25957, 4063, 1010, 2070, 1997, 2029, 5681, 2572, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


在`encode_plus`函数之上，还有`batch_encode_plus`函数（类似地，在`decode`之上，还有`batch_decode`

&emsp; 两者参数类似，**`batch_encode_plus`函数针对批量文本`batch_text`**，**或者批量句对`text_pairs`**

在针对批量文本`batch_text`的示例中，注意`batch_encode_plus`函数返回字典中的`attention_mask`字段

&emsp; 可以发现，**`attention_mask`字段通过`01`标注出词素序列中该位置是否为补零**，可以用做自注意力的掩模

In [12]:
batch_text = ["a series of escapades demonstrating the adage that",
              "what is good for the goose is also good for the gander",
              "some of which occasionally amuses",
              "but none of which amounts to much of a story" ]

encoded = tokenizer.batch_encode_plus(batch_text_or_text_pairs=batch_text, padding=True,
                                      add_special_tokens=True, max_length=16, truncation=True, 
                                      return_attention_mask=True)
print(encoded)

{'input_ids': [[101, 1037, 2186, 1997, 9686, 17695, 18673, 14313, 1996, 15262, 3351, 2008, 102, 0, 0], [101, 2054, 2003, 2204, 2005, 1996, 13020, 2003, 2036, 2204, 2005, 1996, 25957, 4063, 102], [101, 2070, 1997, 2029, 5681, 2572, 25581, 102, 0, 0, 0, 0, 0, 0, 0], [101, 2021, 3904, 1997, 2029, 8310, 2000, 2172, 1997, 1037, 2466, 102, 0, 0, 0]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0]]}


而在针对批量句对`text_pairs`的示例中，注意`batch_encode_plus`函数返回字典中的`attention_mask`字段

&emsp; 可以发现，**`token_type_ids`字段通过`01`标注出词素序列中该位置为句对中的第几句**，句对用`[SEP]`分割

In [13]:
text_pairs = [("a series of escapades demonstrating the adage that",
               "what is good for the goose is also good for the gander"),
              ("some of which occasionally amuses",
               "but none of which amounts to much of a story")]

encoded = tokenizer.batch_encode_plus(batch_text_or_text_pairs=text_pairs, padding=True,
                                      add_special_tokens=True, max_length=32, truncation=True, 
                                      return_attention_mask=True)
print(encoded)

{'input_ids': [[101, 1037, 2186, 1997, 9686, 17695, 18673, 14313, 1996, 15262, 3351, 2008, 102, 2054, 2003, 2204, 2005, 1996, 13020, 2003, 2036, 2204, 2005, 1996, 25957, 4063, 102], [101, 2070, 1997, 2029, 5681, 2572, 25581, 102, 2021, 3904, 1997, 2029, 8310, 2000, 2172, 1997, 1037, 2466, 102, 0, 0, 0, 0, 0, 0, 0, 0]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0]]}


回到`encode_plus`上，在接下来的示例中，**使用内置的`functools.partial`模块构造`encode`函数**

&emsp; 接着**使用该函数对`databundle`进行数据预处理**，由于`tokenizer.encode_plus`返回的是一个字典

&emsp; 读入的是一个字段，所以此处使用`apply_field_more`方法，得到结果自动并入`databundle`中如下

In [14]:
from functools import partial

encode = partial(tokenizer.encode_plus, max_length=32, truncation=True,
                 return_attention_mask=True)
print(encode)

data_bundle.apply_field_more(encode, field_name='text', progress_bar='tqdm')
print(data_bundle.datasets['train'])

functools.partial(<bound method PreTrainedTokenizerBase.encode_plus of PreTrainedTokenizer(name_or_path='bert-base-uncased', vocab_size=30522, model_max_len=512, is_fast=False, padding_side='right', special_tokens={'bos_token': '[BOS]', 'eos_token': '[EOS]', 'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})>, max_length=32, truncation=True, return_attention_mask=True)


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

Processing:   0%|          | 0/2 [00:00<?, ?it/s]

+------------------+----------+-----+------------------+--------------------+--------------------+
| text             | label    | len | input_ids        | token_type_ids     | attention_mask     |
+------------------+----------+-----+------------------+--------------------+--------------------+
| ['this', 'qui... | positive | 11  | [101, 2023, 4... | [0, 0, 0, 0, 0,... | [1, 1, 1, 1, 1,... |
| ['a', 'comedy... | positive | 19  | [101, 1037, 1... | [0, 0, 0, 0, 0,... | [1, 1, 1, 1, 1,... |
| ['a', 'positi... | neutral  | 26  | [101, 1037, 1... | [0, 0, 0, 0, 0,... | [1, 1, 1, 1, 1,... |
| ['a', 'series... | negative | 37  | [101, 1037, 2... | [0, 0, 0, 0, 0,... | [1, 1, 1, 1, 1,... |
+------------------+----------+-----+------------------+--------------------+--------------------+


### 2.3 补充：GloVe 词嵌入的使用

如何使用传统的GloVe词嵌入